<a href="https://colab.research.google.com/github/rashid54/SWE_428-ML_assignment/blob/main/final_lab_assignment/problemset_01_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm
import copy
import zipfile


from google.colab import drive
drive.mount('/content/drive')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mounted at /content/drive


In [2]:
classes = ('AbdomenCT', 'ChestCT', 'CXR', 'Hand', 'HeadCT')
batch_size = 128
num_classes = len(classes)

In [4]:
# zip_file_name = '/CNN_dataset.zip'
# zip_parent_dir = '/content/drive/MyDrive/academics/4_2/'
# zip_file_dir = zip_parent_dir + zip_file_name
# extract_folder_dir = zip_file_dir + 'extract'

# extract the zipped file
!unzip drive/MyDrive/academics/4_2/CNN_dataset.zip

Streaming output truncated to the last 5000 lines.
 extracting: CNN_dataset/CXR/007830.jpeg  
 extracting: CNN_dataset/CXR/004808.jpeg  
 extracting: CNN_dataset/CXR/005493.jpeg  
 extracting: CNN_dataset/CXR/007104.jpeg  
 extracting: CNN_dataset/CXR/001142.jpeg  
 extracting: CNN_dataset/CXR/006524.jpeg  
 extracting: CNN_dataset/CXR/008251.jpeg  
 extracting: CNN_dataset/CXR/005458.jpeg  
 extracting: CNN_dataset/CXR/001716.jpeg  
 extracting: CNN_dataset/CXR/008743.jpeg  
 extracting: CNN_dataset/CXR/006586.jpeg  
 extracting: CNN_dataset/CXR/004909.jpeg  
 extracting: CNN_dataset/CXR/006811.jpeg  
 extracting: CNN_dataset/CXR/009564.jpeg  
 extracting: CNN_dataset/CXR/005829.jpeg  
 extracting: CNN_dataset/CXR/006153.jpeg  
 extracting: CNN_dataset/CXR/007234.jpeg  
 extracting: CNN_dataset/CXR/007258.jpeg  
 extracting: CNN_dataset/CXR/008746.jpeg  
 extracting: CNN_dataset/CXR/001212.jpeg  
 extracting: CNN_dataset/CXR/007942.jpeg  
 extracting: CNN_dataset/CXR/001569.jpeg  
 ex

In [5]:
# Data loading
data_dir = '/content/CNN_dataset'

normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    normalize
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

# create the ImageFolder dataset
dataset = datasets.ImageFolder(root= data_dir)

# split the dataset into training, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# apply the transforms to each set
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = valid_transform
test_dataset.dataset.transform = valid_transform

train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True,
                                           pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                         pin_memory=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False, 
                                          pin_memory=True)


In [6]:
print(len(train_dataset), " ", len(val_dataset), " ", len(test_dataset))

40000   5000   5000


In [7]:
class ConvNeuralNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        # self.layer3 = nn.Sequential(
        #     nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=0),
        #     nn.BatchNorm2d(128),
        #     nn.ReLU(),
        #     nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(53*53*64, 128)
        self.dropout = nn.Dropout(p=0.2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(128, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [8]:
def train(model, criterion, optimizer, scheduler):
  total_step = len(train_loader)
  losses = 0.0
  correct = 0
  total = 0
  model.train() # switch mode

  for i, (images, labels) in tqdm(enumerate(train_loader)):  
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images) #object() is shorthand for object.__call__()
      _, predicted = torch.max(outputs.data, 1)
      loss = criterion(outputs, labels)
        
      optimizer.zero_grad()  # Zero the gradients
      loss.backward() # Compute the gradients
      # Update the parameters
      optimizer.step() # Perform the update

      losses += loss.item()*images.size(0)           
      total += labels.size(0)
      correct += (predicted == labels).sum().item()      
     
  epoch_loss = losses/total
  epoch_acc = 100*(correct/total)
  return epoch_acc, epoch_loss

In [9]:
def validate(model, criterion, optimizer, scheduler):
  total_step = len(val_loader)
  model.eval() # switch mode

  with torch.no_grad():
    losses = 0.0
    correct = 0
    total = 0
    for i, (images, labels) in tqdm(enumerate(val_loader)):  
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        losses += loss.item()*images.size(0)           
        total += labels.size(0)
        correct += (predicted == labels).sum().item()    
    
    epoch_loss = losses/total
    epoch_acc = 100*(correct/total)
    return epoch_acc, epoch_loss

In [10]:
learning_rate = 0.001

model = ConvNeuralNet(num_classes).to(device)
model = model.to(device)
# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 
                             lr=learning_rate,
                             momentum=0.8, 
                             weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)

In [11]:
num_epochs = 20
best_val_accuracy = 0.0
patience = 10
early_stopping_counter = 0
start_time = time.time()

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    train_epoch_acc, train_epoch_loss = train(model, criterion, optimizer, scheduler)
    print ('Train Accuracy: {:.4f}, Loss: {:.4f}' 
                    .format(train_epoch_acc, train_epoch_loss ))
    val_epoch_acc, val_epoch_loss = validate(model, criterion, optimizer, scheduler)
    print ('Val Accuracy: {:.4f}, Loss: {:.4f}' 
                    .format(val_epoch_acc, val_epoch_loss))
    
    if val_epoch_acc > best_val_accuracy:
      best_val_accuracy = val_epoch_acc
      early_stopping_counter = 0
      best_model_weights = copy.deepcopy(model.state_dict())
    else:
      early_stopping_counter+=1
    
    # Stop the training if the validation accuracy has not improved for `patience` epochs
    if early_stopping_counter >= patience:
      print("Early stopping at epoch {}".format(epoch + 1))
      break

end_time = time.time() 
time_elapsed = end_time - start_time
print('Total time needed for training: {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
model.load_state_dict(best_model_weights)

Epoch 0/19
----------


313it [02:44,  1.90it/s]


Train Accuracy: 97.9925, Loss: 0.0871


40it [00:13,  2.99it/s]


Val Accuracy: 99.6200, Loss: 0.0128
Epoch 1/19
----------


...

313it [02:40,  1.96it/s]


Train Accuracy: 99.9925, Loss: 0.0008


40it [00:13,  2.96it/s]

Val Accuracy: 99.9000, Loss: 0.0024
Total time needed for training: 57m 50s


<All keys matched successfully>

In [12]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
  
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        # print(images.shape)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) #torch.max(outputs.data, 1) function returns a tuple (values, indices) 
        total += labels.size(0)
        correct += (predicted == labels).sum().item() # converts the resulting scalar tensor to a Python scalar using the item() method

    print('Accuracy of the model on test images: {} %'.format(100 * correct / total))

Accuracy of the model on test images: 99.86 %


In [13]:
with torch.no_grad():
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    for images, labels in test_loader:
        # print(images.shape)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) #torch.max(outputs.data, 1) function returns a tuple (values, indices) 
        for label, prediction in zip(labels, predicted):
          if label == prediction:
            correct_pred[classes[label]] += 1
          total_pred[classes[label]] += 1 

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: AbdomenCT is 100.0 %
Accuracy for class: ChestCT is 99.7 %
Accuracy for class: CXR   is 100.0 %
Accuracy for class: Hand  is 99.6 %
Accuracy for class: HeadCT is 100.0 %
